In [26]:
from sys import platform
import warnings
import fastf1 as ff1
from fastf1 import plotting
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd

if platform == "win32":
    path = 'C:/Users/olive/GitHub/F1-Fantasy-Predictor/'
elif platform == "darwin":
    path = '~/Documents/GitHub/F1-Fantasy-Predictor/'

warnings.filterwarnings("ignore", category=RuntimeWarning)
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [27]:
# Enable the cache
ff1.Cache.enable_cache(path+'data/cache') 
# ff1.Cache.disabled()

In [28]:
df = pd.DataFrame(columns=['season','round','driver', 'time', 'lapNum', 's1', 's2', 's3', 'spI1', 'spI2', 'SpeedFL',
       'spst', 'isPB', 'compound', 'tyrelife', 'isFresh', 'Stint',
       'trackStatus', 'isAccurate'])

In [29]:
year = 2021
for circuit in range(1,23):
    # Load the session data
    session = ff1.get_session(year, circuit, 'FP3')
    session.load()
    laps = session.load_laps(with_telemetry=True)

    # laps = session.laps
    laps = laps[[
    'DriverNumber', 'LapTime', 'LapNumber',
    'Sector1Time', 'Sector2Time', 'Sector3Time',
    'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
    'Compound', 'TyreLife', 'FreshTyre', 'Stint',
    'TrackStatus', 'IsAccurate'
    ]]

    laps = laps.rename(columns={'DriverNumber': 'driver','LapTime': 'time','LapNumber': 'lapNum','Sector1Time': 's1','Sector2Time': 's2','Sector3Time': 's3','SpeedI1': 'spI1','SpeedI2': 'spI2','SpeedST': 'spst', 'IsPersonalBest': 'isPB','Compound': 'compound','TyreLife': 'tyrelife','FreshTyre': 'isFresh','TrackStatus': 'trackStatus','IsAccurate': 'isAccurate',})

    laps = laps[laps['time'].notnull()]

    laps['time'] = laps['time'].apply(lambda x: x.total_seconds())
    laps['s1'] = laps['s1'].apply(lambda x: x.total_seconds())
    laps['s2'] = laps['s2'].apply(lambda x: x.total_seconds())
    laps['s3'] = laps['s3'].apply(lambda x: x.total_seconds())

    fastest_laps = laps.query('isPB == True').sort_values('time')
    fastest_laps['season'] = year
    fastest_laps['round'] = circuit

    df = pd.concat([df, fastest_laps])

core           INFO 	Loading data for Bahrain Grand Prix - Practice 3 [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/fastf1/core.py:1492: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for key, value in row.iteritems():
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
/Library/Frameworks/Python

ValueError: Session type 'FP3' does not exist for this event

In [30]:
df

,season,round,driver,time,lapNum,s1,s2,s3,spI1,spI2,SpeedFL,spst,isPB,compound,tyrelife,isFresh,Stint,trackStatus,isAccurate
77,2021,1,33,90.577,8,28.648,39.201,22.728,244.0,268.0,286.0,317.0,True,SOFT,2.0,True,4,1,True
63,2021,1,44,91.316,8,28.722,39.801,22.793,239.0,264.0,290.0,320.0,True,SOFT,2.0,True,3,1,True
171,2021,1,10,91.583,9,28.951,39.680,22.952,242.0,270.0,289.0,321.0,True,SOFT,2.0,True,3,1,True
257,2021,1,77,91.855,12,28.982,39.931,22.942,235.0,260.0,289.0,320.0,True,SOFT,2.0,True,3,1,True
160,2021,1,11,91.908,9,29.106,40.010,22.792,242.0,267.0,288.0,317.0,True,SOFT,2.0,True,3,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,2021,9,3,65.725,16,16.702,29.185,19.838,310.0,248.0,284.0,305.0,True,SOFT,2.0,True,4,1,True
357,2021,9,7,65.747,27,16.724,29.141,19.882,312.0,243.0,283.0,311.0,True,SOFT,6.0,False,5,1,True
91,2021,9,47,66.078,19,16.746,29.468,19.864,313.0,245.0,281.0,309.0,True,SOFT,2.0,True,3,1,True
367,2021,9,6,66.105,6,16.741,29.387,19.977,312.0,245.0,282.0,312.0,True,SOFT,2.0,True,5,1,True


In [4]:
# laps = session.laps
laps = laps[[
    'DriverNumber', 'LapTime', 'LapNumber',
    'Sector1Time', 'Sector2Time', 'Sector3Time',
    'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
    'Compound', 'TyreLife', 'FreshTyre', 'Stint',
    'TrackStatus', 'IsAccurate'
    ]]

laps = laps.rename(columns={'DriverNumber': 'driver','LapTime': 'time','LapNumber': 'lapNum','Sector1Time': 's1','Sector2Time': 's2','Sector3Time': 's3','SpeedI1': 'spI1','SpeedI2': 'spI2','SpeedST': 'spst', 'IsPersonalBest': 'isPB','Compound': 'compound','TyreLife': 'tyrelife','FreshTyre': 'isFresh','TrackStatus': 'trackStatus','IsAccurate': 'isAccurate',})

laps = laps[laps['time'].notnull()]

In [5]:
laps['time'] = laps['time'].apply(lambda x: x.total_seconds())
laps['s1'] = laps['s1'].apply(lambda x: x.total_seconds())
laps['s2'] = laps['s2'].apply(lambda x: x.total_seconds())
laps['s3'] = laps['s3'].apply(lambda x: x.total_seconds())

In [6]:
fastest_laps = laps.query('isPB == True').sort_values('time')
fastest_laps['season'] = year
fastest_laps['round'] = circuit

In [10]:
laps.columns

Index(['driver', 'time', 'lapNum', 's1', 's2', 's3', 'spI1', 'spI2', 'SpeedFL',
       'spst', 'isPB', 'compound', 'tyrelife', 'isFresh', 'Stint',
       'trackStatus', 'isAccurate'],
      dtype='object')

In [149]:
laps.sort_values('time')

,driver,time,lapNum,s1,s2,s3,spI1,spI2,SpeedFL,spst,isPB,compound,tyrelife,isFresh,Stint,trackStatus,isAccurate
138,33,71.294,16,18.740,33.516,19.038,222.0,201.0,270.0,280.0,True,SOFT,5.0,True,2,1,True
140,33,71.309,18,18.760,33.489,19.060,219.0,203.0,268.0,279.0,False,SOFT,7.0,True,2,1,True
74,55,71.341,21,18.760,33.585,18.996,221.0,203.0,270.0,278.0,True,SOFT,5.0,True,3,2,True
78,55,71.363,25,18.771,33.519,19.073,220.0,203.0,269.0,279.0,False,SOFT,9.0,False,4,1,True
67,55,71.452,14,18.775,33.571,19.106,220.0,207.0,267.0,277.0,False,SOFT,14.0,True,1,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,16,110.264,20,33.863,46.606,29.795,123.0,144.0,268.0,161.0,False,SOFT,3.0,True,3,2,True
134,33,111.668,12,51.650,38.511,21.507,196.0,181.0,264.0,246.0,False,SOFT,1.0,True,2,1,False
68,55,112.269,15,22.221,38.006,52.042,159.0,181.0,NaN,253.0,False,SOFT,15.0,True,1,1,False
62,55,112.623,9,29.630,53.161,29.832,116.0,96.0,262.0,201.0,False,SOFT,9.0,True,1,1,True


In [150]:
laps.sort_values('time').query('compound == "SOFT" & tyrelife <= 3.0').groupby(['driver']).agg({
    'time': 'mean'
}).sort_values('time')

,time
driver,
63,80.384667
10,80.996000
3,81.120500
11,83.028000
14,83.109000
55,83.668500
77,83.951200
7,85.124000
6,85.380333
